In [ ]:
pip install pandas-profiling

In [ ]:
!pip install tweepy
!pip install wordcloud
!pip install textblob

In [ ]:
import pandas_profiling
import tweepy                   
from google.colab import drive  
import json
import csv
from datetime import date
from datetime import datetime
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sb
import re
import time
import string
import warnings

# for all NLP related operations on text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from wordcloud import WordCloud
import matplotlib.animation as animation
import operator
import plotly.express as px
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


# To consume Twitter's API
import tweepy
from tweepy import OAuthHandler 

# To identify the sentiment of text
from textblob import TextBlob

# ignoring all the warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# downloading stopwords corpus
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stopwords = set(stopwords.words("english"))

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
drive.mount('/content/gdrive')
path = './gdrive/MyDrive/WDPS_data'

Mounted at /content/gdrive


In [ ]:
tweets_df = pd.read_json('/content/gdrive/MyDrive/WDPS_Data/ElectiontweetsFinal.json', lines=True)

In [ ]:
len(tweets_df.index)

79832

In [ ]:
tweets_df.head(10)

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/Politic_Life/status/133138...,2020-11-24 23:54:28+00:00,Billet #1 ~ #USElections2020 \nMaintenant que ...,Billet #1 ~ #USElections2020 \nMaintenant que ...,1331385727926591496,"{'username': 'Politic_Life', 'displayname': 'L...",[],[],1,0,0,0,1331385727926591496,fr,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,NaN,None,None
1,https://twitter.com/PaulMaxit/status/133138476...,2020-11-24 23:50:37+00:00,"#USElections2020 #JoeBiden pas encore élu, mai...","#USElections2020 #JoeBiden pas encore élu, mai...",1331384760921415680,"{'username': 'PaulMaxit', 'displayname': 'Paul...",[https://www.zerohedge.com/geopolitical/bidens...,[https://t.co/YASYitYqvn],0,10,7,0,1331384760921415680,fr,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
2,https://twitter.com/Johnbenitezjr/status/13313...,2020-11-24 23:42:39+00:00,#barrons #cnbc #bloomberg #stocks #money #spor...,#barrons #cnbc #bloomberg #stocks #money #spor...,1331382753787461632,"{'username': 'Johnbenitezjr', 'displayname': '...",[https://twitter.com/ReggieMiddleton/status/13...,[https://t.co/XjH9Mw52ZE],1,0,1,0,1331382753787461632,und,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,{'url': 'https://twitter.com/ReggieMiddleton/s...,None
3,https://twitter.com/vjmahon/status/13313732824...,2020-11-24 23:05:00+00:00,"“Finally, don’t bothsides this. The decay of U...","“Finally, don’t bothsides this. The decay of U...",1331373282474409985,"{'username': 'vjmahon', 'displayname': '💧Vince...",[https://messaging-custom-newsletters.nytimes....,[https://t.co/7J1BQgQAb1],0,0,0,0,1331373282474409985,en,"<a href=""http://twitter.com/#!/download/ipad"" ...",http://twitter.com/#!/download/ipad,Twitter for iPad,None,NaN,None,"[{'username': 'paulkrugman', 'displayname': 'P..."
4,https://twitter.com/PaulMaxit/status/133137197...,2020-11-24 22:59:49+00:00,#USElections2020 #JoeBiden nous concocte un pr...,#USElections2020 #JoeBiden nous concocte un pr...,1331371977643323394,"{'username': 'PaulMaxit', 'displayname': 'Paul...",[https://www.nationalreview.com/corner/bidens-...,[https://t.co/mRavqfFew9],1,11,8,0,1331371977643323394,fr,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
5,https://twitter.com/PaulMaxit/status/133136861...,2020-11-24 22:46:27+00:00,#USElections2020 À la lecture de ce grotesque ...,#USElections2020 À la lecture de ce grotesque ...,1331368612880379914,"{'username': 'PaulMaxit', 'displayname': 'Paul...",[https://www.lefigaro.fr/international/rudy-gi...,[https://t.co/rjF9Z0XVzg],0,6,7,0,1331368612880379914,fr,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
6,https://twitter.com/Yohanne17456752/status/133...,2020-11-24 22:38:53+00:00,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,1331366708179660800,"{'username': 'Yohanne17456752', 'displayname':...",[https://twitter.com/awolallo/status/133133195...,[https://t.co/6rCZ9g7d44],0,0,0,0,1331366708179660800,und,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,None,NaN,{'url': 'https://twitter.com/awolallo/status/1...,None
7,https://twitter.com/Yohanne17456752/status/133...,2020-11-24 22:38:01+00:00,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,1331366489882914816,"{'username': 'Yohanne17456752', 'displayname':...",[https://twitter.com/awolallo/status/133105236...,[https://t.co/329cQUYXeL],0,0,0

In [ ]:
tweets_df['content'].isna().sum()

0

In [ ]:
tweets_df['id'].isna().sum()

0

In [ ]:
#The tweets scraped provide a lot of information. For our project, we are only interested in the tweets.
#Create a new dataframe with tweets.
column = tweets_df[["renderedContent"]]
TweetText_df = column.copy()

In [ ]:
TweetText_df.head(10)

,renderedContent
0,Billet #1 ~ #USElections2020 \nMaintenant que ...
1,"#USElections2020 #JoeBiden pas encore élu, mai..."
2,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U..."
4,#USElections2020 #JoeBiden nous concocte un pr...
5,#USElections2020 À la lecture de ce grotesque ...
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
#Determine Sentiment count of the dataframe.
def fetch_sentiment_using_textblob(text):
    analysis = TextBlob(text)
    # set sentiment 
    if analysis.sentiment.polarity >= 0:
        return 'positive'
    else: 
        return 'negative'

In [ ]:
sentiments_using_textblob = TweetText_df.renderedContent.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))
pd.DataFrame(sentiments_using_textblob.value_counts())

,renderedContent
positive,71153
negative,8679


In [ ]:
#Assign ID
TweetText_df['sentiment'] = sentiments_using_textblob
TweetText_df['id'] = TweetText_df.index + 1
TweetText_df.head()

,renderedContent,sentiment,id
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3
3,"“Finally, don’t bothsides this. The decay of U...",positive,4
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5


In [ ]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text 

In [ ]:
# Create new column 'refinedTweets' to store cleaned up tweets.
TweetText_df['refinedTweets'] = np.vectorize(remove_pattern)(TweetText_df['renderedContent'], "@[\w]*: | *RT*")
TweetText_df.head(10)

,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet #1 ~ #USElections2020 \nMaintenant que ...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,"#USElections2020 #JoeBiden pas encore élu, mai..."
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,"“Finally, don’t bothsides this. The decay of U..."
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections2020 #JoeBiden nous concocte un pr...
5,#USElections2020 À la lecture de ce grotesque ...,negative,6,#USElections2020 À la lecture de ce grotesque ...
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,9,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,positive,10,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
# Remove @ and RT
TweetText_df['refinedTweets'] = np.vectorize(remove_pattern)(TweetText_df['renderedContent'], "@[\w]*: | *RT*")
TweetText_df.head(10)

,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet #1 ~ #USElections2020 \nMaintenant que ...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,"#USElections2020 #JoeBiden pas encore élu, mai..."
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,"“Finally, don’t bothsides this. The decay of U..."
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections2020 #JoeBiden nous concocte un pr...
5,#USElections2020 À la lecture de ce grotesque ...,negative,6,#USElections2020 À la lecture de ce grotesque ...
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,9,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,positive,10,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
#Remove hyperlinks contained in tweets.
cleanTweets = []

for index, row in TweetText_df.iterrows():
    words_without_links = [word for word in row.refinedTweets.split() if 'http' not in word]
    cleanTweets.append(' '.join(words_without_links))

TweetText_df['refinedTweets'] = cleanTweets
TweetText_df.head(10)

,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet #1 ~ #USElections2020 Maintenant que le...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,"#USElections2020 #JoeBiden pas encore élu, mai..."
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,"“Finally, don’t bothsides this. The decay of U..."
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections2020 #JoeBiden nous concocte un pr...
5,#USElections2020 À la lecture de ce grotesque ...,negative,6,#USElections2020 À la lecture de ce grotesque ...
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,9,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,positive,10,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
TweetText_df = TweetText_df[TweetText_df['refinedTweets']!='']
TweetText_df.head()

,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet #1 ~ #USElections2020 Maintenant que le...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,"#USElections2020 #JoeBiden pas encore élu, mai..."
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,"“Finally, don’t bothsides this. The decay of U..."
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections2020 #JoeBiden nous concocte un pr...


In [ ]:
TweetText_df.drop_duplicates(subset=['refinedTweets'], keep=False)
TweetText_df.head()

,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet #1 ~ #USElections2020 Maintenant que le...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,"#USElections2020 #JoeBiden pas encore élu, mai..."
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,"“Finally, don’t bothsides this. The decay of U..."
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections2020 #JoeBiden nous concocte un pr...


In [ ]:
TweetText_df = TweetText_df.reset_index(drop=True)
TweetText_df.head(10)

,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet #1 ~ #USElections2020 Maintenant que le...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,"#USElections2020 #JoeBiden pas encore élu, mai..."
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,"“Finally, don’t bothsides this. The decay of U..."
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections2020 #JoeBiden nous concocte un pr...
5,#USElections2020 À la lecture de ce grotesque ...,negative,6,#USElections2020 À la lecture de ce grotesque ...
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,9,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,positive,10,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
pip install unidecode

In [ ]:
import unidecode

In [ ]:
#Removing unnecessary characters using unidecode
TweetText_df['refinedTweets'] = TweetText_df['refinedTweets'].apply(lambda x : ' '.join([unidecode.unidecode(word) for word in x.split()]))
TweetText_df.head(10)


,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet #1 ~ #USElections2020 Maintenant que le...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,"#USElections2020 #JoeBiden pas encore elu, mai..."
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,"""Finally, don't bothsides this. The decay of U..."
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections2020 #JoeBiden nous concocte un pr...
5,#USElections2020 À la lecture de ce grotesque ...,negative,6,#USElections2020 A la lecture de ce grotesque ...
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,9,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,positive,10,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
TweetText_df['refinedTweets'] = TweetText_df['refinedTweets'].str.replace("[^a-zA-Z# ]", "")
TweetText_df.head(10)

,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet # #USElections Maintenant que les resu...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,#USElections #JoeBiden pas encore elu mais dej...
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,Finally dont bothsides this The decay of US de...
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections #JoeBiden nous concocte un progra...
5,#USElections2020 À la lecture de ce grotesque ...,negative,6,#USElections A la lecture de ce grotesque arti...
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,9,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,positive,10,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
# Perform stopword removal.
# filter out all the stopwords.
# Create list of tuples containing stopwords and the sentimentType.
stopwords = set(stopwords)
cleanTweets = []

for index, row in TweetText_df.iterrows():
    
    words_without_stopwords = [word for word in row.refinedTweets.split() if not word in stopwords]
    cleanTweets.append(' '.join(words_without_stopwords))

TweetText_df['refinedTweets'] = cleanTweets
TweetText_df.head(10)

,renderedContent,sentiment,id,refinedTweets
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet # #USElections Maintenant que les resul...
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,#USElections #JoeBiden pas encore elu mais dej...
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,Finally dont bothsides The decay US democracy ...
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections #JoeBiden nous concocte un progra...
5,#USElections2020 À la lecture de ce grotesque ...,negative,6,#USElections A la lecture de ce grotesque arti...
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,9,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,positive,10,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
#Create a new column to find hashtags. Later on, we can use this column to count the number and determine the popularity of a hashtag.
TweetText_df['#'] = TweetText_df['refinedTweets'].apply(lambda x : ' '.join([word for word in x.split() if word.startswith('#')]))
TweetText_df.head(10)

,renderedContent,sentiment,id,refinedTweets,#
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet # #USElections Maintenant que les resul...,# #USElections # #JoeBiden
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,#USElections #JoeBiden pas encore elu mais dej...,#USElections #JoeBiden
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,Finally dont bothsides The decay US democracy ...,#USElections
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections #JoeBiden nous concocte un progra...,#USElections #JoeBiden #JoeBiden
5,#USElections2020 À la lecture de ce grotesque ...,negative,6,#USElections A la lecture de ce grotesque arti...,#USElections
6,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
7,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
8,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,positive,9,#Ethiopia #Tigray #War #IStandWithTigray #IamT...,#Ethiopia #Tigray #War #IStandWithTigray #IamT...
9,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,positive,10,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...,#FakeCoronavirus #CoronaHoax #ArrestBillGates ...


In [ ]:
frame = TweetText_df['#']

In [ ]:
frame = pd.DataFrame(frame)

In [ ]:
frame = frame.rename({'#':'Count(#)'},axis = 'columns')
frame[frame['Count(#)'] == ''] = 'No hashtags'
frame.head()

,Count(#)
0,# #USElections # #JoeBiden
1,#USElections #JoeBiden
2,#barrons #cnbc #bloomberg #stocks #money #spor...
3,#USElections
4,#USElections #JoeBiden #JoeBiden


In [ ]:
HTC_TweetText_df = pd.concat([TweetText_df,frame],axis = 1)

In [ ]:
HTC_TweetText_df.drop('#',axis = 1,inplace = True)
HTC_TweetText_df.head()

,renderedContent,sentiment,id,refinedTweets,Count(#)
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,Billet # #USElections Maintenant que les resul...,# #USElections # #JoeBiden
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,#USElections #JoeBiden pas encore elu mais dej...,#USElections #JoeBiden
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barrons #cnbc #bloomberg #stocks #money #spor...,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,Finally dont bothsides The decay US democracy ...,#USElections
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#USElections #JoeBiden nous concocte un progra...,#USElections #JoeBiden #JoeBiden


In [ ]:
#Perform Lemmitization.
lemmatizer = WordNetLemmatizer()
HTC_TweetText_df['refinedTweets'] = HTC_TweetText_df['refinedTweets'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [ ]:
#Peform Stemming.
ps = PorterStemmer()
HTC_TweetText_df['refinedTweets'] = HTC_TweetText_df['refinedTweets'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))

In [ ]:
from nltk.corpus import stopwords

In [ ]:
len(HTC_TweetText_df)

79832

In [ ]:
#Perform Tokenization.
corpus = []
for i in range(0,79832):
    tweet = HTC_TweetText_df['refinedTweets'][i]
    tweet = tweet.lower()
    tweet = tweet.split()
    tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

In [ ]:
#Collect hashtags based on their sentiments

hash_positive = []
hash_negative = []

In [ ]:
def hashtag_extract(x):
    hashtags = []
    # Loop over the words in the tweet
    for i in x:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)

    return hashtags

In [ ]:
hash_positive = hashtag_extract(HTC_TweetText_df['refinedTweets'][HTC_TweetText_df['sentiment'] == 'positive'])

# extracting hashtags from racist/sexist tweets
hash_negative = hashtag_extract(HTC_TweetText_df['refinedTweets'][HTC_TweetText_df['sentiment'] == 'negative'])

# Converting a multidimensional list to a 1-D list
hash_positive = sum(hash_positive,[])
hash_negative = sum(hash_negative,[])

In [ ]:
q = Counter(hash_positive)
q = dict(q.most_common())
l_positive_count = list(q.values())
l_positive_count[0:4]

[81600, 13279, 9762, 6646]

In [ ]:
r = Counter(hash_negative)
r = dict(r.most_common())
l_negative_count = list(r.values())
l_negative_count[0:4]

[9821, 1172, 1045, 551]

In [ ]:
l_positive_values = list(q.keys())
l_positive_values[0:4]

['uselect', 'elect', 'trump', 'electionday']

In [ ]:
l_negative_values = list(r.keys())
l_negative_values[0:4]

['uselect', 'elect', 'trump', 'biden']

In [ ]:
#Creating a dataframe to represent top 20 positive and negative hash words
l1 = pd.DataFrame(l_positive_values[0:20],columns = ['Positive_Words'])
l2 = pd.DataFrame(l_positive_count[0:20],columns = ['Positive_Count'])
l3 = pd.DataFrame(l_negative_values[0:20],columns = ['Negative_Words'])
l4 = pd.DataFrame(l_negative_count[0:20],columns = ['Negative_Count'])
z = pd.concat([l1,l2,l3,l4],axis = 1)
z

,Positive_Words,Positive_Count,Negative_Words,Negative_Count
0,uselect,81600,uselect,9821
1,elect,13279,elect,1172
2,trump,9762,trump,1045
3,electionday,6646,biden,551
4,biden,6026,usaelect,428
5,joebiden,5826,electionday,397
6,usaelect,4816,joebiden,353
7,uspresidentialelect,3953,bidenharri,353
8,donaldtrump,3719,uspresidentialelect,319
9,bidenharri,3306,donaldtrump,293


In [ ]:
#Animated plot for positive words with their frequency
fig = px.bar(z, x="Positive_Words", y="Positive_Count",animation_frame="Positive_Count",
            hover_name="Positive_Words")
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1200
fig.show()


In [ ]:
# Animated plot for negative words with their frequency
fig = px.bar(z, x="Negative_Words", y="Negative_Count",animation_frame="Negative_Count",
            hover_name="Negative_Words")
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1200
fig.show()

In [ ]:
#Normal histogram of positive words
fig = px.bar(z, x="Positive_Words", y="Positive_Count",
            hover_name="Positive_Words",color = 'Positive_Count')
fig.show()

In [ ]:
#Normal histogram of negative words
fig = px.bar(z, x="Negative_Words", y="Negative_Count",
            hover_name="Negative_Words",color= 'Negative_Count')
fig.show()

In [ ]:
HTC_TweetText_df.head()

,renderedContent,sentiment,id,refinedTweets,Count(#)
0,Billet #1 ~ #USElections2020 \nMaintenant que ...,positive,1,billet # #uselect mainten que le resultat offi...,# #USElections # #JoeBiden
1,"#USElections2020 #JoeBiden pas encore élu, mai...",positive,2,#uselect #joebiden pa encor elu mai deja vendu...,#USElections #JoeBiden
2,#barrons #cnbc #bloomberg #stocks #money #spor...,positive,3,#barron #cnbc #bloomberg #stock #money #sport ...,#barrons #cnbc #bloomberg #stocks #money #spor...
3,"“Finally, don’t bothsides this. The decay of U...",positive,4,final dont bothsid the decay US democraci isnt...,#USElections
4,#USElections2020 #JoeBiden nous concocte un pr...,positive,5,#uselect #joebiden nou concoct un programm eco...,#USElections #JoeBiden #JoeBiden


In [ ]:
HTC_TweetText_df.to_csv('/content/gdrive/MyDrive/WDPS_Data/cleanedElectiontweetsFinal', index = False)

In [ ]:
HTC_TweetText_df.to_json('/content/gdrive/MyDrive/WDPS_Data/cleanedElectiontweetsFinalJSON')